In [3]:
from langchain_ollama.llms import OllamaLLM
import sqlite3

llm = OllamaLLM(model="llama3:latest", temperature=0.7)
# response = llm.invoke("Hello, how are you?")
# print(response)


In [4]:
from langchain.tools import tool

@tool
def query_database(query: str) -> str:
    """Run a SQL query on the company database."""
    try:
        conn = sqlite3.connect("company.db")
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        return str(rows)
    except Exception as e:
        return f"Error: {e}"


In [5]:
from langchain.agents import initialize_agent

tools = [query_database]

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    verbose=True
)


C:\Users\rezva\AppData\Local\Temp\ipykernel_8152\3979333104.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [6]:
response = agent_executor.invoke({"input": "List all employees in Engineering department."})
print("👨‍💻 Engineering Employees:", response["output"])



> Entering new AgentExecutor chain...
Thought: To list all employees in the Engineering department, I'll need to run a query that filters employees by their department.

Action: query_database
Action Input: "SELECT * FROM employees WHERE department = 'Engineering';"
Observation: [(1, 'Ali', 'Engineering', 8000), (3, 'Reza', 'Engineering', 9000)]
Thought:Here's the continuation:

Thought: Now that I have the list of employees in the Engineering department, I can extract the necessary information.

Action: query_database
Action Input: "SELECT name FROM employees WHERE department = 'Engineering';"
Observation: [('Ali',), ('Reza',)]
Thought:Let's continue from where we left off:

Question: What are the names of all employees in Engineering department?

Thought: Since I already have the list of employees in the Engineering department, I can extract their names by running another query.

Action: query_database
Action Input: "SELECT name FROM (SELECT * FROM employees WHERE department = 'Eng

In [7]:
# سوال ۲: میانگین حقوق کارمندان
response = agent_executor.invoke({"input": "What is the average salary of employees?"})
print("💰 Average Salary:", response["output"])



> Entering new AgentExecutor chain...
Let's get started.

Thought: To find the average salary of employees, I'll need to query the database for all employee salaries and then calculate the average. I'll use a SQL query to retrieve the necessary information.

Action: query_database
Action Input: "SELECT AVG(salary) FROM employees;"
Observation: [(7833.333333333333,)]
Thought:Here's my attempt at answering the question:

Question: What is the average salary of employees?
Thought: Let's get started.
Thought: To find the average salary of employees, I'll need to query the database for all employee salaries and then calculate the average. I'll use a SQL query to retrieve the necessary information.

Action: query_database
Action Input: "SELECT AVG(salary) FROM employees;"
Observation: [(7833.333333333333,)]
Thought:Here's my attempt at answering the question:

Final Answer: The average salary of employees is $7833.33.

Note that I'm assuming the result returned by the database query is a s

In [ ]:
# سوال ۳: پروژه‌هایی با ددلاین در اکتبر
response = agent_executor.invoke({"input": "Show all projects with deadlines in October."})
print("📅 October Projects:", response["output"])



> Entering new AgentExecutor chain...
Thought: To show all projects with deadlines in October, I need to run a SQL query on our company database. The query should filter results to only include projects with deadlines that fall within the month of October.

Action: query_database
Action Input: "SELECT * FROM projects WHERE deadline MONTH = '10'"
Observation: Error: near "MONTH": syntax error
Thought:Let's think this through!

Thought: The error message suggests that there's a syntax issue with the query. I'm not familiar with the specific SQL dialect used by our company database, but in standard SQL, the MONTH function is not used to extract the month from a date.

Action: query_database
Action Input: "SELECT * FROM projects WHERE deadline >= '2023-10-01' AND deadline < '2024-11-01'"
Observation: []
Thought: